In [1]:
import sqlite3
import json
from pprint import pprint

def list_and_dictify(items, one=False):
    if items is None:
        return None
    
    elif one:
        return dict(items)

    new_list = []
    for row in items:
        new_list.append(dict(row))

    return new_list


conn = sqlite3.connect("data.db")
conn.row_factory = sqlite3.Row

cur = conn.cursor()
cur.execute("SELECT * FROM receipts")
receipts = list_and_dictify(items=cur.fetchall())

cur.execute(
    '''
    SELECT receipts.id AS receipt_id,
           store, item, price, quantity
    FROM receipts_junction
    JOIN receipts ON receipt_id=receipts.id
    '''
)
items = list_and_dictify(cur.fetchall())


In [2]:
def group_by_id(id, items):
    new_list = []
    for i, item in enumerate(items):
        if item["receipt_id"] == id:
            new_list.append(items.pop(i))

    return new_list


full_list = group_by_id(33, items)


def create_full(receipts, items):
    for i, receipt in enumerate(receipts):
        new_list = group_by_id(receipt["id"], items)
        receipts[i]["products"] = new_list

    return receipts

full_list = create_full(receipts, items)
#pprint(full_list)

with open("dumped_data", "w") as f:
    if True:
        f.write(json.dumps(full_list))